# PD Genetic Landscape Plot Data Processing
- **Author** - Frank Grenn
- **Date Started** - November 2020
- **Quick Description:** code to get statistics for plot

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
DATADIR='/path/to/data'

## 1) Get List of Risk Variants
may need to modify code depending on input files

In [ ]:
risk_vars = pd.DataFrame()

### a) Nalls et al. 2019

In [ ]:
meta5_loci = pd.read_csv(f"{DATADIR}/META5Loci.csv")
meta5_loci['GWAS']='META5'
meta5_loci['EFFECT_FREQ_NALLS']=meta5_loci['EFFECT_FREQ']
meta5_loci['BETA_NALLS']=meta5_loci['BETA']
meta5_loci['P_NALLS']=meta5_loci['P']
meta5_loci = meta5_loci[['RSID','LOC_NUM','CHR_BP','NEAR_GENE','GWAS','REF','ALT','EFFECT_ALLELE','OTHER_ALLELE','EFFECT_FREQ_NALLS','BETA_NALLS','P_NALLS']]
print(meta5_loci.shape)
print(meta5_loci.head())

### b) Iwaki et al. 2019

In [ ]:
prog_loci = pd.read_csv(f"{DATADIR}/ProgressionLoci.csv")
prog_loci['GWAS']='Progression'
prog_loci['EFFECT_ALLELE']=prog_loci['ALT']
prog_loci['OTHER_ALLELE']=prog_loci['REF']
prog_loci = prog_loci.rename(columns={"MAF": "FREQ"})
prog_loci['EFFECT_FREQ_IWAKI']=prog_loci['FREQ']
prog_loci['BETA_IWAKI']=prog_loci['BETA']
prog_loci['P_IWAKI']=prog_loci['P']
prog_loci = prog_loci[['RSID','LOC_NUM','CHR_BP','NEAR_GENE','GWAS','REF','ALT','EFFECT_ALLELE','OTHER_ALLELE','EFFECT_FREQ_IWAKI','BETA_IWAKI','P_IWAKI']]

print(prog_loci.shape)
print(prog_loci.head())

### c) Foo et al. 2020

In [ ]:
asian_loci = pd.read_csv(f"{DATADIR}/AsianLoci.csv")
asian_loci['GWAS']='Asian'
asian_loci['EFFECT_ALLELE']=asian_loci['ALT']
asian_loci['OTHER_ALLELE']=asian_loci['REF']
asian_loci = asian_loci.rename(columns={"MAF": "FREQ"})
asian_loci['EFFECT_FREQ_FOO']=asian_loci['FREQ']
asian_loci['BETA_FOO']=asian_loci['BETA']
asian_loci['P_FOO']=asian_loci['P']
asian_loci = asian_loci[['RSID','LOC_NUM','CHR_BP','NEAR_GENE','GWAS','REF','ALT','EFFECT_ALLELE','OTHER_ALLELE','EFFECT_FREQ_FOO','BETA_FOO','P_FOO']]
print(asian_loci.shape)
print(asian_loci.head())

combine all variants into one df

In [ ]:
risk_vars = meta5_loci.append(prog_loci).append(asian_loci)
print(risk_vars.shape)
print(risk_vars.head())

In [ ]:
risk_vars.tail()

## 2) Get Effect Sizes (Beta/Odds Ratio), Frequencies, and P values from GWASes
do this for the full list of risk variants. 

also check we are using statistics for the correct allele

### a) Nalls et al. 2019

In [ ]:
meta5_gwas = pd.read_csv(f"{DATADIR}/meta5_sumstats_harmonized.csv")
print(meta5_gwas.shape)
print(meta5_gwas.head())

In [ ]:
meta5_gwas_sub = meta5_gwas[['RSID','REF','ALT','A1','A2','BETA','FREQ','P']]

In [ ]:
merge_meta5_gwas = pd.merge(risk_vars, meta5_gwas_sub, how='left', left_on='RSID', right_on='RSID')
print(merge_meta5_gwas.shape)
print(merge_meta5_gwas.tail())

In [ ]:
print(merge_meta5_gwas.head())

#### now check for the right allele
looks like EFFECT_ALLELE should match A1

In [ ]:
meta5_gwas_match=merge_meta5_gwas[merge_meta5_gwas['EFFECT_ALLELE']==merge_meta5_gwas['A1']]
print(meta5_gwas_match.shape)
print(meta5_gwas_match.head())

In [ ]:
meta5_gwas_mismatch=merge_meta5_gwas[merge_meta5_gwas['EFFECT_ALLELE']!=merge_meta5_gwas['A1']]
print(meta5_gwas_mismatch.shape)
print(meta5_gwas_mismatch.tail())

adjust the effect size and frequency for the mismatched variants

In [ ]:
meta5_gwas_mismatch['BETA']=-1*meta5_gwas_mismatch['BETA']
meta5_gwas_mismatch['FREQ']=1-meta5_gwas_mismatch['FREQ']

In [ ]:
final_meta5_gwas = meta5_gwas_match.append(meta5_gwas_mismatch)

print(final_meta5_gwas.shape)
print(final_meta5_gwas.head())

In [ ]:
print(final_meta5_gwas.tail())

In [ ]:
final_meta5_gwas['OR'] = np.exp(final_meta5_gwas['BETA'])

In [ ]:
final_meta5_gwas = final_meta5_gwas[['RSID','LOC_NUM','CHR_BP','NEAR_GENE','GWAS','EFFECT_ALLELE','OTHER_ALLELE','FREQ','BETA','OR','P']]
final_meta5_gwas.columns = ['RSID','LOC_NUM','CHR_BP','NEAR_GENE','GWAS','EFFECT_ALLELE','OTHER_ALLELE','EFFECT_FREQ_NALLS','BETA_NALLS','OR_NALLS','P_NALLS']

### b) Foo et al. 2020 

In [ ]:
asian_gwas = pd.read_csv(f"{DATADIR}/asiangwas_sumstats_harmonized.csv")
print(asian_gwas.shape)
print(asian_gwas.head())


In [ ]:
asian_gwas_sub = asian_gwas[['RSID','A1','A2','BETA','FREQ','P']]

In [ ]:
merge_asian_gwas = pd.merge(risk_vars, asian_gwas_sub, how='left', left_on='RSID', right_on='RSID')
print(merge_asian_gwas.shape)
print(merge_asian_gwas.tail())

In [ ]:
merge_asian_gwas

#### now check for the right allele
need to look at beta since we don't have freqs, looks like asian gwas A1 should match EFFECT_ALLELE

In [ ]:
asian_gwas_match=merge_asian_gwas[merge_asian_gwas['EFFECT_ALLELE']==merge_asian_gwas['A1']]
print(asian_gwas_match.shape)
print(asian_gwas_match.head())

In [ ]:
asian_gwas_mismatch=merge_asian_gwas[merge_asian_gwas['EFFECT_ALLELE']!=merge_asian_gwas['A1']]
print(asian_gwas_mismatch.shape)
print(asian_gwas_mismatch.tail())

In [ ]:
asian_gwas_mismatch['BETA']=-1*asian_gwas_mismatch['BETA']

In [ ]:
final_asian_gwas = asian_gwas_match.append(asian_gwas_mismatch)
print(final_asian_gwas.shape)
print(final_asian_gwas.head())

In [ ]:
final_asian_gwas['OR'] = np.exp(final_asian_gwas['BETA'])

In [ ]:
final_asian_gwas = final_asian_gwas[['RSID','LOC_NUM','CHR_BP','NEAR_GENE','GWAS','EFFECT_ALLELE','OTHER_ALLELE','FREQ','BETA','OR','P']]
final_asian_gwas.columns = ['RSID','LOC_NUM','CHR_BP','NEAR_GENE','GWAS','EFFECT_ALLELE','OTHER_ALLELE','EFFECT_FREQ_FOO','BETA_FOO','OR_FOO','P_FOO']

## c) combine

In [ ]:
gwas_stats = pd.merge(left = final_meta5_gwas, right = final_asian_gwas, on = ['RSID','LOC_NUM','CHR_BP','NEAR_GENE','GWAS','EFFECT_ALLELE','OTHER_ALLELE'])
print(gwas_stats.shape)
print(gwas_stats.head())

add progression loci

In [ ]:
gwas_stats = pd.merge(left = gwas_stats, right = prog_loci, on = ['RSID','LOC_NUM','CHR_BP','NEAR_GENE','GWAS','EFFECT_ALLELE','OTHER_ALLELE'], how = 'left')
print(gwas_stats.shape)
print(gwas_stats.head())
print(gwas_stats.tail())

In [ ]:
gwas_stats['OR_IWAKI'] = np.exp(gwas_stats['BETA_IWAKI'])

In [ ]:
gwas_stats[gwas_stats['GWAS']=='Progression']

## 3) Get Population Frequencies Using ANNOVAR

create ANNOVAR input file

In [ ]:
avinput = risk_vars[['CHR_BP','REF','ALT']]

In [ ]:
split= avinput['CHR_BP'].str.split(":",expand = True)
avinput['CHR']=split[0]
avinput['BP']=split[1]

In [ ]:
avinput = avinput[['CHR','BP','BP','REF','ALT']]
print(avinput.shape)
print(avinput.head())

In [ ]:

avinput.to_csv(f"{DATADIR}/gwas_risk_variants.avinput",index=None,sep=' ')

write ANNOVAR script

In [ ]:
with open(f"{DATADIR}/get_frequencies_annovar.sh","w") as bash_file:
    bash_file.write(f'''#!/bin/bash\n\
module load annovar\n\
annotate_variation.pl --filter --build hg19 --dbtype gnomad211_genome --buildver hg19 --otherinfo {DATADIR}/gwas_risk_variants.avinput $ANNOVAR_DATA/hg19''')
bash_file.close()

run

In [ ]:
print(f"sbatch {DATADIR}/get_frequencies_annovar.sh")

read

In [ ]:
freqs = pd.read_csv(f"{DATADIR}/gwas_risk_variants.avinput.hg19_gnomad211_genome_dropped",sep="\t",header=None)
freqs.columns = ['db','freqs','snp']
print(freqs.shape)
print(freqs.head())

columns:

gnomad211_genome: Chr Start End Ref Alt AF AF_popmax AF_male AF_female AF_raw AF_afr AF_sas AF_amr AF_eas AF_nfe AF_fin AF_asj AF_oth non_topmed_AF_popmax non_neuro_AF_popmax non_cancer_AF_popmax controls_AF_popmax

In [ ]:
freqs[['CHR','START','END','REF','ALT']]=freqs.snp.str.split(" ",expand=True)
print(freqs.head())

In [ ]:
freqs[['AF','AF_popmax','AF_male','AF_female','AF_raw','AF_afr','AF_sas','AF_amr','AF_eas','AF_nfe','AF_fin','AF_asj','AF_oth','non_topmed_AF_popmax','non_neuro_AF_popmax','non_cancer_AF_popmax','controls_AF_popmax']]=freqs.freqs.str.split(",",expand=True)
print(freqs.head())

In [ ]:
freqs['CHR_BP'] = freqs['CHR'].astype(str)+":"+freqs['START'].astype(str)
freqs = freqs.drop(labels=['freqs','snp','db','CHR','START','END'],axis=1)
print(freqs.head())

for now only want nfe, eas, and afr


In [ ]:
freqs = freqs[['CHR_BP','REF','ALT','AF_nfe','AF_eas','AF_afr']]
print(freqs.head())

now compare alleles to make sure we are using the right frequencies

In [ ]:
merged = pd.merge(gwas_stats, freqs, left_on = 'CHR_BP', right_on = 'CHR_BP',how = 'inner')
print(merged.shape)

In [ ]:
merged.head()

In [ ]:
merged.tail()

looks like EFFECT_ALLELE should match ALT

In [ ]:
match=merged[merged['EFFECT_ALLELE']==merged['ALT_y'].str.upper()]
print(match.shape)
print(match.head())

In [ ]:
mismatch=merged[merged['EFFECT_ALLELE']!=merged['ALT_y'].str.upper()]
print(mismatch.shape)
print(mismatch.head())

In [ ]:
mismatch['AF_nfe']=1-mismatch['AF_nfe'].astype(float)
mismatch['AF_eas']=1-mismatch['AF_eas'].astype(float)
mismatch['AF_afr']=1-mismatch['AF_afr'].astype(float)

In [ ]:
final_pop_freqs = match.append(mismatch)

In [ ]:
final_pop_freqs = final_pop_freqs.drop(labels=['REF_y','ALT_y','REF_x','ALT_x'],axis=1)

In [ ]:
print(final_pop_freqs.shape)
print(final_pop_freqs.head())
print(final_pop_freqs.tail())

## 4) Make P_GWAS Column
P_GWAS column will contain the P value for the variant from the GWAS it is from. That way we can use this column as the p value to display in the plot.

In [ ]:
final_pop_freqs['P_GWAS']=0

In [ ]:
final_pop_freqs.loc[final_pop_freqs['GWAS']=='META5','P_GWAS'] = final_pop_freqs.loc[final_pop_freqs['GWAS']=='META5','P_NALLS']
final_pop_freqs.loc[final_pop_freqs['GWAS']=='Asian','P_GWAS'] = final_pop_freqs.loc[final_pop_freqs['GWAS']=='Asian','P_FOO']
final_pop_freqs.loc[final_pop_freqs['GWAS']=='Progression','P_GWAS'] = final_pop_freqs.loc[final_pop_freqs['GWAS']=='Progression','P_IWAKI']

In [ ]:
final_pop_freqs.to_csv(f"/path/to/app/PDLandscapePlot/www/risk_variant_data.csv",index=None)


## 5) Make GWAS_ref Column
to store text for the GWAS to be displayed


In [ ]:
final_pop_freqs['GWAS_ref']=''

In [ ]:
final_pop_freqs.loc[final_pop_freqs['GWAS']=='META5','GWAS_ref'] = "Nalls et al. 2019"
final_pop_freqs.loc[final_pop_freqs['GWAS']=='Asian','GWAS_ref'] = "Foo et al. 2020"
final_pop_freqs.loc[final_pop_freqs['GWAS']=='Progression','GWAS_ref'] = "Iwaki et al. 2019"

In [ ]:
final_pop_freqs.to_csv(f"/path/to/app/PDLandscapePlot/www/risk_variant_data.csv",index=None)